In [1]:
from benchmark import Benchmark, benchmark_pandas
from quantum import H_from_qubo, Qobj
from MaxCut import MaxCut

from qutip.parallel import parallel_map, serial_map

from qaoa_and_tdvp import QAOA, QAOAResult, qaoa_tdvp_rhs, tdvp_optimize_qaoa, scipy_optimize

import pickle

import numpy as np
from numpy.typing import NDArray
from typing import List, Tuple, Union, Optional, Callable, Any
import networkx as nx
import matplotlib.pyplot as plt
import plotly as py

import pandas as pd

In [2]:
n = 4
with open(f"./instances/n{n}_instances.p","rb") as f:
    instances = pickle.load(f)
delta = tuple(1 for _ in range(2 * n))
df_instances = pd.DataFrame(data={"instance":instances, "p":1}, columns=["instance",
                "qaoa",
                "p",
                "n",
                "delta_0",
                "tollarance",
                "algorithm",
                "res",
                "delta",
                "path",
                "steps",
                "num_fun_calls",
                "real duration",
                "message",])

In [3]:
df_instances

,instance,qaoa,p,n,delta_0,tollarance,algorithm,res,delta,path,steps,num_fun_calls,real duration,message
0,<MaxCut.MaxCut object at 0x7f122c2fbe80>,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,<MaxCut.MaxCut object at 0x7f1231051240>,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,<MaxCut.MaxCut object at 0x7f11bc226590>,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,<MaxCut.MaxCut object at 0x7f11bc227730>,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,<MaxCut.MaxCut object at 0x7f11bc226bf0>,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,<MaxCut.MaxCut object at 0x7f11bc226950>,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
x = benchmark_pandas(instance=instances[0], p=1, tollarance=.1, optimizers={"tdvp":False, "scipy":True, "gradient_descent":False})
x

Running benchmark on instance with 4 with optimizers <generator object benchmark_pandas.<locals>.<genexpr> at 0x7f11b99c4e40>
optimizing with scipy
Done Scipy_optim



,instance,qaoa,p,n,delta_0,tollarance,algorithm,res,delta,path,steps,num_fun_calls,real duration,message
0,<MaxCut.MaxCut object at 0x7f11bc119d80>,<qaoa_and_tdvp.QAOA object at 0x7f11b9397af0>,1,4,"(1, 1)",0.1,scipy,\n scipy_cobyla terminated with sucess ...,"(0.7967443681366325, 0.356162038224893)","[(1, 1), (2.0, 1.0), (1.0, 2.0), (0.3165890092...",None,13,0.130807,Optimization terminated successfully.


,a,b
0,0,0
1,5,6
2,15,16


In [30]:
df_instances.apply(
    lambda x:benchmark_pandas(instance=x['instance'], p=x['p'], tollarance=.1, optimizers={"tdvp":False, "scipy":True, "gradient_descent":False}),
    axis=1,
)



Running benchmark on instance with 4 with optimizers <generator object benchmark_pandas.<locals>.<genexpr> at 0x7f11b99c7a00>
optimizing with scipy
Done Scipy_optim

Running benchmark on instance with 4 with optimizers <generator object benchmark_pandas.<locals>.<genexpr> at 0x7f11b99c6e30>
optimizing with scipy
Done Scipy_optim

Running benchmark on instance with 4 with optimizers <generator object benchmark_pandas.<locals>.<genexpr> at 0x7f11b99c45f0>
optimizing with scipy
Done Scipy_optim

Running benchmark on instance with 4 with optimizers <generator object benchmark_pandas.<locals>.<genexpr> at 0x7f11b99c6b20>
optimizing with scipy
Done Scipy_optim

Running benchmark on instance with 4 with optimizers <generator object benchmark_pandas.<locals>.<genexpr> at 0x7f11b99c45f0>
optimizing with scipy
Done Scipy_optim

Running benchmark on instance with 4 with optimizers <generator object benchmark_pandas.<locals>.<genexpr> at 0x7f11b99c7530>
optimizing with scipy
Done Scipy_optim



0                                       instance  \...
1                                       instance  \...
2                                       instance  \...
3                                       instance  \...
4                                       instance  \...
5                                       instance  \...
dtype: object

In [19]:
df

0                                             0    ...
1                                             0    ...
2                                             0    ...
3                                             0    ...
4                                             0    ...
5                                             0    ...
dtype: object

In [69]:
pd.Dat([{"a":5, 'b':6},{"a":15, 'b':16}])

1 2 3


In [3]:
Delta = .1
min_p = 1
max_p = 8

In [5]:
for n in range(4, 7):
    with open(f"./instances/n{n}_instances.p","rb") as f:    
        instances = pickle.load(f)
    delta = tuple(1 for _ in range(2 * n))
    bench = Benchmark()

    for num, i in enumerate(instances):
        print(f"instance {num}/{len(instances)}")
        for p in range(min_p, max_p+1):
            print(f"\t p = {p}/{max_p}")
            bench.run(
                qaoa=QAOA(i.qubo,p=p),
                delta_0=tuple(1 for _ in range(2 * p)),
                tdvp_stepsize=1,
                tdvp_grad_tol=1e-3,
                p=p,
                tdvp_lineq_solver="RK45",
            )
            bench.save(f"RK45_n{n}_p-1-10_Delta-1_benchmarks.p")

instance 0/6
	 p = 1/8
donestep 30

Done Scipy_optim

	 p = 2/8
donestep 37

Done Scipy_optim

	 p = 3/8
donestep 45

Done Scipy_optim

	 p = 4/8
donestep 65

Done Scipy_optim

	 p = 5/8
donestep 78

Done Scipy_optim

	 p = 6/8
donestep 59

Done Scipy_optim

	 p = 7/8
